# Transcription et Résumé d'une Réunion

Ce notebook vous permet de :

1. **Transcrire** un fichier audio de réunion en texte grâce à la version open source de Whisper.
2. **Générer un résumé** de la transcription ainsi que **les points clés** via l'API OpenAI (GPT).

## Pré-requis

- Un compte Google pour utiliser Google Colab.
- Avoir enregistré votre fichier audio (exemple : `Meeting.m4a`) dans votre Google Drive.
- Votre clé API OpenAI (pour les résumés et l'extraction des points clés).
- **TOUJOURS VERIFIER** que le mode d'éxecution est GPU T4, pour le vérifier, faites :

 onglet **Execution** --> **Modifier le type d'execution** --> **Accélérateur de matériel** --> **GPU T4**

Suivez les instructions de chaque cellule pour monter votre Google Drive, renseigner votre clé API et indiquer le chemin de votre fichier audio.

**Pour lancer une cellule, cliquez une seule fois sur le bouton play en haut à gauche des cellules contenant du code.**


In [ ]:
# Monter Google Drive pour accéder à vos fichiers
from google.colab import drive
drive.mount('/content/drive')

# Installer les packages nécessaires
# Pour Whisper (si non déjà installé)
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt update > /dev/null 2>&1 && sudo apt install -y ffmpeg > /dev/null 2>&1
!pip install -q openai

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


**Note :** La première exécution de la cellule vous demandera d'autoriser l'accès à votre Google Drive. Cliquez sur le lien, connectez-vous et accepter les différentes étapes. (autorisez tout)

### Chargement clé API

Lancez juste la cellule ci-dessous. Si une demande d'accès est demandé, autorisez la.

In [ ]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openapi_key

**Important :** La clé API est secrète mais accessible avec vos droits. Merci de ne pas partager directement ce drive à quelqu'un d'extérieur à la famille.

### Corps de l'IA

Lancez juste la cellule.

In [ ]:
import whisper
from openai import OpenAI

# Récupérer la clé API depuis la variable d'environnement
openapi_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openapi_key)

# Chargement du modèle Whisper open source (version "turbo")
# Vous pouvez changer le modèle si besoin (ex. "base", "small", "medium", etc.)
model = whisper.load_model("turbo")

def speech_converter(audio_file):
    """
    Transcrit l'audio en texte en utilisant le modèle open source Whisper.
    """
    result = model.transcribe(audio_file, language='fr')
    return result["text"]

def send_prompt(messages, model="gpt-4o-mini", timeout=30):
    """
    Envoie une requête à l'API OpenAI pour générer un résumé ou extraire les points clés.
    """
    completion = client.chat.completions.create(
        messages=messages,
        model=model,
        timeout=timeout
    )
    response = completion.choices[0].message.content
    return response

# Prompts pour le résumé et l'extraction des points clés
PROMPT1 = '''Tu es un agent chargé de résumer la transcription d'une réunion qui t'a été donnée.
La réunion est en français. Tu dois faire en sorte de résumer le plus fidèlement possible et de retranscrire le contexte.
S'agissant de réunions professionnelles, veille à réduire au maximum la perte d'information lors de ce résumé.'''

PROMPT2 = '''Tu es un agent chargé d'extraire les points clés d'une transcription écrite de réunion.
Ton objectif est d'extraire tous les points clés des textes qui vont t'être donnés.
S'agissant de réunions professionnelles, veille à ne pas rater de points clés.'''

def get_gpt_response(query, prompt):
    """
    Prépare et envoie la requête à l'API OpenAI pour obtenir la réponse souhaitée.
    """
    chat_api = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": query}
    ]
    response = send_prompt(chat_api)
    return response

def SpeechExtractor(audio_file):
    """
    Extrait la transcription, le résumé et les points clés d'un fichier audio.
    """
    transcription = speech_converter(audio_file)
    resume = get_gpt_response(transcription, PROMPT1)
    keypoints = get_gpt_response(transcription, PROMPT2)
    return transcription, resume, keypoints

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


### **Chargement du fichier**

**Indiquez ici le chemin vers votre fichier audio sur Google Drive.**

Pour récupérer le chemin :
1. Allez dans le panneau de gauche ("Fichiers") de Colab.
2. Cliquez sur l'icône Google Drive.
3. Naviguez jusqu'à votre fichier audio ("drive" -> "MyDrive" -> "IA pour maman"), faites un clic droit et sélectionnez "Copier le chemin d'accès".
4. Bien mettre le chemin d'accès entre guillemets (simple ou double peut importe)


**Exemple :**


audio_file = '/content/drive/MyDrive/IA pour maman/Meeting.m4a'

In [ ]:
audio_file ='/content/drive/MyDrive/IA pour maman/Nouvel enregistrement 16.m4a'

### **Lancement de l'IA**

**Lancer la cellule ci-dessous pour que l'IA fasse le traitement de l'audio et le résumé de celui-ci.**



**Attention :** Vérifier bien que le bon fichier audio est sélectionné ci-dessus.


**En cas d'erreur ou autre**, vous pouvez arrêter le traitement en appuyant sur le carré qui a remplacé le 'play' en haut à gauche de la cellule.**

.

**Pour convertir un autre fichier, remplacer le fichier ci-dessu par le chemin d'accès de votre nouveau fichier puis relancer la cellule ci-dessous**
  
.

.


**Note importante** : Pour le résultat, pour faciliter l'extraction, vous pouvez au choix copier coller directement la sortie, soit cliquez sur le bouton en haut à gauche de la sortie (petite flèche) et sélectionner afficher en plein écran pour n'avoir que la sortie en visuel (echap pour en sortir).

In [ ]:
# Exécuter l'extraction : transcription, résumé et points clés
transcription, resume, keypoints = SpeechExtractor(audio_file)

print("------------------------------------------------------------")
print("Transcription :\n")
print(transcription)
print("\n------------------------------------------------------------\n")
print("Résumé :\n")
print(resume)
print("\n------------------------------------------------------------\n")
print("Points clés :\n")
print(keypoints)
print("\n------------------------------------------------------------")

------------------------------------------------------------
Transcription :

 Il y avait toujours trois mois par mois, le dernier NGVO au mois d'août, on nous a indiqué la question. Donc nous avons posé une question légitime. La médiation littorale, ça démarre, c'était une réunion par mois, insuffisant. Donc aujourd'hui, c'est ce qui est donné comme raison, c'est une raison d'organisation de personnes. Il y a deux personnes qui vont avoir un heureux événement, mais elles ne sont pas là, donc le système est arrêté. Et deux personnes en moins, on est obligé de faire une réunion par mois. Même notre adjointe, Mme Robert, a constaté la chose, on s'est rapprochés de Mme Bonnet, et puis finalement, comme c'est la ville de Marseille qui organise ça et qui réserve les personnes personnelles, voilà où on en est. Je parle de la médiation littorale du 7e, je pense que c'est pour les paroles. Donc voilà, ça, ça serait un truc qu'il faudrait faire remonter aussi, mais que c'est déjà remonter d'aut

Vous disposez désormais d'un notebook complet.
Instructions d'utilisation :

Montez votre Google Drive avec la cellule 2.

1. Installez les dépendances (Whisper et OpenAI).  
2. Définissez votre clé API dans la cellule 3.  
3. Collez le code de la cellule 4 pour charger le modèle et définir les fonctions.  
4. Indiquez le chemin vers votre fichier audio dans la cellule 5.  
5. Lancez la cellule 6 pour obtenir la transcription, le résumé et les points clés.

Bon usage !
